# training ecapa-tdnn with 16k

In [ ]:
!CUDA_VISIBLE_DEVICES=1 python trainECAPAModel.py \
    --sampling_rate 16000 \
    --eval_list /workspace/data/chgo/veri_test2.txt\
    --save_path /workspace/data/chgo/exps16k/exp1\
    --mode blip \
    --loss aam_infonce \
    --model ecapa-tdnn
    #--initial_model /workspace/data/chgo/voxceleb_code/ECAPA-TDNN-main_8k/exps/

# training ecapa-tdnn with 8k

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python trainECAPAModel.py \
    --sampling_rate 8000 \
    --eval_list /workspace/data/chgo/veri_test2.txt\
    --save_path /workspace/data/chgo/exps8k/exp1\
    --mode blip \
    --loss aam_infonce \
    --model ecapa-tdnn

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python trainECAPAModel.py \
    --eval \
    --sampling_rate 16000 \
    --eval_list /workspace/data/chgo/list_test_all2.txt \
    --initial_model /workspace/data/chgo/exps8k/exp1/model/model_0061.model

In [ ]:
!CUDA_VISIBLE_DEVICES=1 python trainECAPAModel.py \
    --eval \
    --sampling_rate 8000 \
    --eval_list /workspace/data/chgo/list_test_all2.txt \
    --initial_model /workspace/data/chgo/voxceleb_code/ECAPA-TDNN-main/exps/model8k.model

In [ ]:
!CUDA_VISIBLE_DEVICES=1 python trainECAPAModel.py \
    --eval \
    --snorm \
    --sampling_rate 16000 \
    --eval_list /workspace/data/chgo/veri_test2.txt \
    --train_list /workspace/data/chgo/train_vox2.txt \
    --train_path /workspace/data/chgo/voxceleb_code/dev/aac/ \
    --initial_model /workspace/data/chgo/voxceleb_code/ECAPA-TDNN-main/exps/model16k.model 

In [11]:
!CUDA_VISIBLE_DEVICES=1 python trainECAPAModel.py \
    --eval \
    --snorm \
    --sampling_rate 8000 \
    --eval_list /workspace/data/chgo/veri_test2.txt \
    --train_list /workspace/data/chgo/train_vox2.txt \
    --train_path /workspace/data/chgo/voxceleb_code/dev/aac/ \
    --initial_model /workspace/data/chgo/voxceleb_code/ECAPA-TDNN-main/exps/model8k.model 

^C


# extract speaker embedding

In [6]:
import model
import torch
import torch, sys, os, tqdm, numpy, soundfile, time, pickle
import torchaudio.transforms as T
import torch.nn.functional as F

use_cuda = torch.cuda.is_available() and True
device = torch.device("cuda" if use_cuda else "cpu")

speaker_encoder = model.ECAPA_TDNN(512, 16000).to(device)

loaded_state = torch.load('/workspace/data/chgo/exps8k/exp1/model/model_0061.model')
self_state = speaker_encoder.state_dict()
new_state_dict = {name.replace("speaker_encoder.", ""): param for name, param in loaded_state.items()}
new_state_dict.pop('speaker_loss.weight')
speaker_encoder.load_state_dict(new_state_dict)

<All keys matched successfully>

In [3]:
sampling_rate = 16000
audio_path = '/workspace/data/chgo/voxceleb_code/wav/id10530/pvAnCk52wRw/00003.wav'
audio, sr  = soundfile.read(os.path.join(audio_path))

audio = torch.FloatTensor(audio)
resampler = T.Resample(sr, sampling_rate, dtype=audio.dtype)
audio = resampler(audio)
with torch.no_grad():
    speaker_encoder.eval()
    embedding = speaker_encoder(audio.unsqueeze(0).to(device), False)
    embedding = F.normalize(embedding, p=2, dim=1)
print(embedding.shape)

torch.Size([1, 192])


# cosine similarity between positive speakers

In [4]:
sampling_rate = 16000
audio_path1 = '/workspace/data/chgo/voxceleb_code/wav/id10530/pvAnCk52wRw/00003.wav'
audio_path2 = '/workspace/data/chgo/voxceleb_code/wav/id10530/pvAnCk52wRw/00001.wav'

audio1, sr  = soundfile.read(os.path.join(audio_path1))
audio2, sr  = soundfile.read(os.path.join(audio_path2))

audio1 = torch.FloatTensor(audio1)
resampler = T.Resample(sr, sampling_rate, dtype=audio.dtype)
audio1 = resampler(audio1)

audio2 = torch.FloatTensor(audio2)
resampler = T.Resample(sr, sampling_rate, dtype=audio.dtype)
audio2 = resampler(audio2)

with torch.no_grad():
    speaker_encoder.eval()
    embedding_11 = speaker_encoder(audio1.unsqueeze(0).to(device), False)
    embedding_11 = F.normalize(embedding_11, p=2, dim=1)
    
    embedding_21 = speaker_encoder(audio2.unsqueeze(0).to(device), False)
    embedding_21 = F.normalize(embedding_21, p=2, dim=1)

score = F.cosine_similarity(embedding_11.to(device), embedding_21.to(device))

print(score)

tensor([0.7683], device='cuda:0')


# cosine similarity between negative speakers

In [5]:
sampling_rate = 16000
audio_path1 = '/workspace/data/chgo/voxceleb_code/wav/id10530/pvAnCk52wRw/00003.wav'
audio_path2 = '/workspace/data/chgo/voxceleb_code/wav/id10906/wI7dFlBaj-8/00011.wav'

audio1, sr  = soundfile.read(os.path.join(audio_path1))
audio2, sr  = soundfile.read(os.path.join(audio_path2))

audio1 = torch.FloatTensor(audio1)
resampler = T.Resample(sr, sampling_rate, dtype=audio.dtype)
audio1 = resampler(audio1)

audio2 = torch.FloatTensor(audio2)
resampler = T.Resample(sr, sampling_rate, dtype=audio.dtype)
audio2 = resampler(audio2)

with torch.no_grad():
    speaker_encoder.eval()
    embedding_11 = speaker_encoder(audio1.unsqueeze(0).to(device), False)
    embedding_11 = F.normalize(embedding_11, p=2, dim=1)
    
    embedding_21 = speaker_encoder(audio2.unsqueeze(0).to(device), False)
    embedding_21 = F.normalize(embedding_21, p=2, dim=1)

score = F.cosine_similarity(embedding_11.to(device), embedding_21.to(device))

print(score)

tensor([-0.0212], device='cuda:0')
